In [30]:
import os
import requests
import time
import re
import pandas as pd
import numpy as np
import random
import pickle
import time
import os

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

from glob import glob


In [31]:
#projects=pd.DataFrame(columns=['kickstarter_page','funding', 'campaign_dates', 'backers', 'category'])

In [32]:
#setting up Selenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.binary_location = '/usr/bin/google-chrome'

chromedriver = "/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


In [33]:
#kicktraq archive page numbers 1-8000 are available
page_numbers = range(7888,8001)

for page_num in page_numbers:
    #test if file for this page already exists
    filename = 'kicktraq_files/kicktraq_page_num' + str(page_num)
    if os.path.isfile(filename):
        continue
    
    url = 'https://www.kicktraq.com/archive/?page=' + str(page_num)
    print('url is ' + url)
    
    kickstarter_page = []
    funding = []
    campaign_dates = []
    backers = []
    categories = []
    
    driver.get(url)
    
    #define condition to wait for: 'next page' button title should be 'go to pagenum+1'
    next_button_xpath = '//*[@title="go to page {}"]'.format(str(page_num+1))
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
    
    #get html from Selenium into Beautiful Soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    all_projects_on_page = soup.find_all(class_="project first odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

    for project in all_projects_on_page: 
        #if statement tests to see if funding info is available. If it is, we include the project. If not, we don't!
        funding_info = project.find(class_='project-details').find(text=re.compile('Funding: .\d*.\d* of .\d*.\d*'))
        if funding_info != None:
            funding.append(str(funding_info).replace('$',''))

            kickstarter_page.append(str(project.find('h2').find('a')['href']))
            campaign_dates.append(str(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)'))))
            backers.append(str(project.find(class_='project-details').find(text=re.compile('Backers: \d*'))))
            categories.append(str(project.find(class_='project-cat').find('a').text))
            d = {'kickstarter_page': kickstarter_page,'funding': funding,
                 'campaign_dates': campaign_dates, 'backers': backers,
                 'category': categories}
            page_df = pd.DataFrame(d)
    
    page_df.to_csv(filename,index=False, encoding='utf-8')
    
    #projects = pd.concat([projects, page_df])
    
    
    
    time.sleep(10)

driver.close()


url is https://www.kicktraq.com/archive/?page=7888


TimeoutException: Message: 


In [ ]:
#ensure column types are string
#col_types = {'kickstarter_page': 'str','funding': 'str', 'campaign_dates': 'str', 'backers': 'str', 'category': 'str'}
#projects = projects.astype(col_types, copy=False, errors='raise', encoding='utf-8')

In [ ]:
#projects.to_csv('kicktraq_data.csv', index=False)

In [ ]:
#with open('kicktraq_data.pickle', 'wb') as f:
#    pickle.dump(projects,f)

In [ ]:
#convert df to numpy array and save in hdf5 format
#proj2 = np.array(proj2)
#h_file = h5py.File('kicktraq_data.h5', 'w')
#h_file.create_dataset('projects_array', data=proj2)
#hf.close()